Import required modules and load the file. The first lines are displayed.

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

path = "../../Data/iris.csv"
data = pd.read_csv(path, delimiter=',')
print(data.head())
print("Empty columns: ", data.columns[data.isnull().any()])

In [ ]:
# Output of colrrelations
correlations = data[data.columns].corr(numeric_only=True)
print('All correlations')
print('-' * 30)
correlations_abs_sum = correlations[correlations.columns].abs().sum()
print(correlations_abs_sum)
print('Weakest correlations')
print('-' * 30)
print(correlations_abs_sum.nsmallest(5))

prepare data

In [ ]:
# possibly drop data['sepal.width'], has low correlation -> test
data = data.drop(['sepal.width'], axis = 1)

# this column is to be predictetd 
col = data['sepal.length']
data = data.drop(['sepal.length'], axis = 1)

# perform OHE
conv_ohe = ['species']
data = pd.get_dummies(data, columns = conv_ohe, dtype=float)

# create object
s_scaler = StandardScaler()
# columns for StandardScaler
cols_to_s_scale = ['petal.length', 'petal.width']
data[cols_to_s_scale] = s_scaler.fit_transform(data[cols_to_s_scale])

In [ ]:
print(data)

build ANN

In [ ]:
# create four tables from the two tables
train_data, test_data, train_col, test_col = train_test_split(data,col, test_size=0.2, random_state=42)

# build ANN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation=tf.nn.relu, input_dim=data.shape[1]),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
# confihure ANN
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

train

In [ ]:
# 70 runs
model.fit(train_data, train_col, epochs=70)

test

In [ ]:
test_loss, test_mae = model.evaluate(test_data, test_col)
print('Test mae:', test_mae)